In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizerFast
#from transformers import AdamW
from torch.optim import AdamW
import json
import pandas as pd
import nltk
import re
import torch
import numpy as np
from torch import nn
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm


model = LongformerForSequenceClassification.from_pretrained('kazzand/ru-longformer-tiny-16384', num_labels=8)
tokenizer = LongformerTokenizerFast.from_pretrained('kazzand/ru-longformer-tiny-16384')


C:\Users\altwayme\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at kazzand/ru-longformer-tiny-16384 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load data from JSON file
with open('articles.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Переведём его в список записей
#    data — это { "1": { "data": {...}}, "2": { "data": {...}}, … }
records = [node['data'] for node in data.values()]

# 3. «Расплющим» вложенные словари в records
df = pd.json_normalize(records)

# Посмотрим
df.head()

,title,authors,publication_date,citation_journals,keywords,anno,url,text,fetch_date,ncr_category,classification
0,Демографические итоги 2014 года. Краткий докла...,Не найдено,2015,Демографическое обозрение,"РОССИЯ, RUSSIA, ДЕМОГРАФИЧЕСКИЕ ИТОГИ, ЧИСЛЕНН...",The Institute of Demography presents a brief v...,https://cyberleninka.ru/article/n/demografiche...,﻿ДЕМОГРАФИЧЕСКИЕ ИТОГИ 2014 ГОДА. КРАТКИЙ ДОКЛ...,2025-04-01,3_Комфортная_безопасная_среда,[1]
1,ГРЕБНЕВАЯ МОДЕЛЬ ДЛЯ ПРОГНОЗА ДЕМОГРАФИЧЕСКИХ ...,"Медведева Елена Ильинична, Крошилин Сергей Вик...",2022,Народонаселение,"СОЦИАЛЬНО-ЭКОНОМИЧЕСКИЕ ПРОБЛЕМЫ, ВОЗРАСТНАЯ С...",The purpose of this study was to use the autho...,https://cyberleninka.ru/article/n/grebnevaya-m...,﻿Э01: 10.19181/рори1айоп.2022.25.2.8\n\nГРЕБНЕ...,2025-04-01,Не определено,[1]
2,ЖЕНЩИНЫ И МУЖЧИНЫ: РАЗЛИЧИЯ В ПОКАЗАТЕЛЯХ РОЖД...,"Архангельский Владимир Николаевич, Калачикова ...",2021,"Экономические и социальные перемены: факты, те...","СТАБИЛЬНОЕ НАСЕЛЕНИЕ, ВОЗРАСТНАЯ МОДЕЛЬ РОЖДАЕ...",The search for the reasons that determine birt...,https://cyberleninka.ru/article/n/zhenschiny-i...,﻿DOI: 10.15838/esc.2021.5.77.10 УДК 314.8:314....,2025-04-01,Не определено,[1]
3,Составляющие демографического потенциала сельс...,"Новиков В.Г., Чалый В.С.",2012,Вестник университета,"СЕЛЬСКИЕ ТЕРРИТОРИИ, ДЕМОГРАФИЯ, РОЖДАЕМОСТЬ, ...","Анализ демографической ситуации, проведенной в...",https://cyberleninka.ru/article/n/sostavlyayus...,СОСТАВЛЯЮЩИЕ ДЕМОГРАФИЧЕСКОГО ПОТЕНЦИАЛА СЕЛЬС...,2025-04-01,Не определено,[1]
4,О прогнозе долгосрочного социально-экономическ...,Не найдено,2014,Финансовая аналитика: проблемы и решения,,По данным Минэкономразвития России публикуются...,https://cyberleninka.ru/article/n/o-prognoze-d...,о прогнозе долгосрочного социально-экономическ...,2025-04-01,1_Сохранение_насел-развитие_здоровье_благополу...,[5]


In [4]:
# Функция проверяет, что все элементы списка — целые числа
def is_numeric_label_list(lbls):
    return isinstance(lbls, list) and all(re.fullmatch(r'\d+', str(x)) for x in lbls)

# Отфильтруем DataFrame
df = df[df['classification'].apply(is_numeric_label_list)].copy()

# Приведём сами метки к спискам int (если нужно дальше считать числами)
df['classification'] = df['classification'].apply(lambda lbls: [int(x) for x in lbls])

# Проверим
print(f"Осталось записей: {len(df)}")
display(df[['text', 'classification']].head())

Осталось записей: 4558


,text,classification
0,﻿ДЕМОГРАФИЧЕСКИЕ ИТОГИ 2014 ГОДА. КРАТКИЙ ДОКЛ...,[1]
1,﻿Э01: 10.19181/рори1айоп.2022.25.2.8\n\nГРЕБНЕ...,[1]
2,﻿DOI: 10.15838/esc.2021.5.77.10 УДК 314.8:314....,[1]
3,СОСТАВЛЯЮЩИЕ ДЕМОГРАФИЧЕСКОГО ПОТЕНЦИАЛА СЕЛЬС...,[1]
4,о прогнозе долгосрочного социально-экономическ...,[5]


In [5]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

def preprocess_text(text: str) -> str:
    # 1) Единообразим 'ё' → 'е' сразу в обоих регистрах
    text = text.replace('Ё','Е').replace('ё','е')

    # 2) Чистим HTML и невидимые символы
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'[\r\n\t]', ' ', text)

    # 3) Оставляем цифры, знаки и **оба регистра** (рус. и англ.)
    text = re.sub(
        r'[^A-Za-z0-9А-Яа-яЕе\-\.,:/%]', 
        ' ', 
        text
    )

    # 4) Нормализуем пробелы
    text = re.sub(r'\s{2,}', ' ', text).strip()

    # 5) Токенизируем и удаляем стоп-слова **независимо от регистра**
    tokens = text.split()
    tokens = [
        t for t in tokens 
        if t.lower() not in russian_stopwords 
           and len(t) > 2
    ]

    # 6) Собираем строку
    return ' '.join(tokens)

df['text_str'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\altwayme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df[['text_str','classification']].head(100)

,text_str,classification
0,ДЕМОГРАФИЧЕСКИЕ ИТОГИ 2014 ГОДА. КРАТКИЙ ДОКЛА...,[1]
1,Э01: 10.19181/рори1айоп.2022.25.2.8 ГРЕБНЕВАЯ ...,[1]
2,DOI: 10.15838/esc.2021.5.77.10 УДК 314.8:314.0...,[1]
3,СОСТАВЛЯЮЩИЕ ДЕМОГРАФИЧЕСКОГО ПОТЕНЦИАЛА СЕЛЬС...,[1]
4,прогнозе долгосрочного социально-экономическог...,[5]
...,...,...
106,Федеральный закон июля 2008 года 123-ФЗ Технич...,"[1, 7]"
107,ДЕМОГРАФИЧЕСКИЕ УСЛОВИЯ ПОВЫШЕНИЯ ПЕНСИОННОГО ...,[0]
108,"Irina Yu. Khovavko, Doct. Sc., Associate profe...",[1]
109,СОЦИАЛЬНО-ДЕМОГРАФИЧЕСКИЕ ПРОБЛЕМЫ РАЗВИТИЯ СЕ...,[1]


In [7]:
# Get all unique labels
unique_labels = set(label for sublist in df['classification'] for label in sublist)

# Create a mapping from label names to integers
label_to_id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}

In [8]:
# Update label preprocessing to handle one-hot encoding
def convert_to_one_hot(labels, num_classes):
    one_hot = torch.zeros(num_classes)
    for label in labels:
        one_hot[label] = 1
    return one_hot

# Apply the one-hot encoding to numeric_labels
num_classes = len(unique_labels)  # Total number of unique labels
df['one_hot_labels'] = df['classification'].apply(lambda x: convert_to_one_hot(x, num_classes))

df['one_hot_labels'].head(1)

0    [tensor(0.), tensor(1.), tensor(0.), tensor(0....
Name: one_hot_labels, dtype: object

In [9]:
# MultilabelStratifiedShuffleSplit
X = df['text_str'].values
Y = np.stack(df['one_hot_labels'].values)

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(msss.split(X, Y))
train_texts, test_texts = X[train_idx], X[test_idx]
train_labels, test_labels = Y[train_idx], Y[test_idx]

In [10]:
# Функция пакетной токенизации текстов и создания тензоров меток
def tokenize_batch(texts, labels):
    # Применяем токенизатор к списку строк
    enc = tokenizer(
        list(texts),
        max_length=16384,       # максимальная длина последовательности
        padding='max_length',   # дополняем до этой длины
        truncation=True,        # обрезаем более длинные тексты
        return_tensors='pt'     # возвращаем PyTorch-тензоры
    )
    # Добавляем в результат тензор меток (float для BCEWithLogitsLoss)
    enc['labels'] = torch.tensor(labels, dtype=torch.float)
    return enc

In [11]:
# Токенизируем тренировочные и валидационные данные
train_enc = tokenize_batch(train_texts, train_labels)
test_enc = tokenize_batch(test_texts, test_labels)

In [12]:
# Кастомный Dataset для работы с уже закодированными данными
class CustomDataset(Dataset):
    def __init__(self, encodings):
        # encodings — словарь с ключами 'input_ids', 'attention_mask', 'labels'
        self.enc = encodings

    def __len__(self):
        # Размер датасета = количество примеров (длина input_ids)
        return len(self.enc['input_ids'])

    def __getitem__(self, idx):
        # Для заданного индекса возвращаем словарь из соответствующих тензоров
        return {key: tensor[idx] for key, tensor in self.enc.items()}

In [13]:
# Создаем объекты Dataset
train_ds = CustomDataset(train_enc)
test_ds   = CustomDataset(test_enc)

In [14]:
# Оборачиваем их в DataLoader для батчевой подачи
train_loader = DataLoader(train_ds, batch_size=3, shuffle=True)  # перемешиваем данные
test_loader   = DataLoader(test_ds,   batch_size=3)               # без перемешивания

In [15]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:  ", torch.version.cuda)
print("Device count:  ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device(), torch.cuda.get_device_name(0))


CUDA available: True
CUDA version:   12.6
Device count:   1
Current device: 0 NVIDIA GeForce RTX 3090


In [16]:
# Подготовка устройства и модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Определяем оптимизатор и функцию потерь
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.BCEWithLogitsLoss()  # для multi-label

# Параметры ранней остановки
best_test_loss = float('inf')
patience = 3
no_improve = 0

# Количество эпох
epochs = 50

for epoch in range(1, epochs + 1):
    # — ТРЕНИРОВКА —
    model.train()
    train_loss = 0.0
    print(f"\n=== Эпоха {epoch}/{epochs} — ТРЕНИРОВКА ===")
    for batch in tqdm(train_loader, desc=" TRAIN", leave=False):
        inputs = {
            'input_ids':      batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(**inputs).logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_loader)
    print(f"ТРЕНИРОВКА → Средний loss: {train_loss:.4f}")

    # — ВАЛИДАЦИЯ —
    model.eval()
    test_loss = 0.0
    print(f"=== Эпоха {epoch}/{epochs} — ВАЛИДАЦИЯ ===")
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="   TEST", leave=False):
            inputs = {
                'input_ids':      batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device)
            }
            labels = batch['labels'].to(device)

            logits = model(**inputs).logits
            test_loss += loss_fn(logits, labels).item()
    test_loss /= len(test_loader)
    print(f"ВАЛИДАЦИЯ → Средний loss: {test_loss:.4f}")

    # — РАННЯЯ ОСТАНОВКА —
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        no_improve = 0
        model.save_pretrained('best_model/')
        tokenizer.save_pretrained('best_model/')
        print("✔ Улучшение! Модель сохранена.")
    else:
        no_improve += 1
        print(f"✖ Без улучшения: {no_improve}/{patience}")
        if no_improve >= patience:
            print("⏹ Ранняя остановка — прекращаем обучение.")
            break



=== Эпоха 1/50 — ТРЕНИРОВКА ===


 TRAIN:   0%|          | 0/1210 [00:00<?, ?it/s]Initializing global attention on CLS token...


ТРЕНИРОВКА → Средний loss: 0.2909
=== Эпоха 1/50 — ВАЛИДАЦИЯ ===


ВАЛИДАЦИЯ → Средний loss: 0.2083
✔ Улучшение! Модель сохранена.

=== Эпоха 2/50 — ТРЕНИРОВКА ===


ТРЕНИРОВКА → Средний loss: 0.1823
=== Эпоха 2/50 — ВАЛИДАЦИЯ ===


...

ВАЛИДАЦИЯ → Средний loss: 0.1870
✖ Без улучшения: 1/3

=== Эпоха 4/50 — ТРЕНИРОВКА ===


ТРЕНИРОВКА → Средний loss: 0.0769
=== Эпоха 4/50 — ВАЛИДАЦИЯ ===


ВАЛИДАЦИЯ → Средний loss: 0.2093
✖ Без улучшения: 2/3

=== Эпоха 5/50 — ТРЕНИРОВКА ===


ТРЕНИРОВКА → Средний loss: 0.0482
=== Эпоха 5/50 — ВАЛИДАЦИЯ ===


ВАЛИДАЦИЯ → Средний loss: 0.2320
✖ Без улучшения: 3/3
⏹ Ранняя остановка — прекращаем обучение.


In [20]:
# Загрузка
model = LongformerForSequenceClassification.from_pretrained('best_model/').to(device)
tokenizer = LongformerTokenizerFast.from_pretrained('best_model/')

# Оценка
from sklearn.metrics import classification_report

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        inputs = {
            'input_ids':      batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }
        labels = batch['labels'].to(device)
        logits = model(**inputs).logits
        preds = (torch.sigmoid(logits) > 0.5).float()

        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

print(classification_report(
    all_labels,
    all_preds,
    target_names=[str(k) for k in label_to_id.keys()],
    zero_division=0
))


              precision    recall  f1-score   support

           0       0.71      0.22      0.33       102
           1       0.84      0.91      0.87       327
           2       0.87      0.85      0.86       177
           3       0.89      0.81      0.85       162
           4       0.97      0.64      0.77        45
           5       0.74      0.68      0.71       133
           6       1.00      0.07      0.13        29
           7       0.66      0.57      0.61       154

   micro avg       0.81      0.72      0.76      1129
   macro avg       0.83      0.60      0.64      1129
weighted avg       0.81      0.72      0.74      1129
 samples avg       0.80      0.77      0.78      1129



In [ ]:
# ---------- 1. загрузка "чистого" корпуса (4558 статей), как раньше ----------
with open(r"articles.json",
          encoding="utf-8") as f:
    raw_main = json.load(f)

main_df = pd.json_normalize(node["data"] for node in raw_main.values())



In [ ]:
# ---------- 2. загрузка ручной выборки --------------------------------------
with open(r"400.json",
          encoding="utf-8") as f:
    raw_val = json.load(f)

val_df = pd.json_normalize(node["data"] for node in raw_val.values())



In [4]:
# ---------- 4. единообразный pre‑processing текста --------------------------
ru_stop = set(stopwords.words("russian"))
def prep(text: str) -> str:
    text = (text or "").replace("Ё", "Е").replace("ё", "е")
    text = re.sub(r"<[^>]+>", " ", text)         # HTML
    text = re.sub(r"[\r\n\t]", " ", text)
    text = re.sub(r"[^A-Za-z0-9А-Яа-яЕе\-,.:/%]", " ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    tokens = [t for t in text.split() if t.lower() not in ru_stop and len(t) > 2]
    return " ".join(tokens)

val_df["text_str"]  = val_df["text"].apply(prep)



In [5]:
# ---------- 5. кодируем метки exactly как в основном корпусе ----------------
def to_int_list(lbls):
    """Возвращает список int‑меток или пустой список."""
    if isinstance(lbls, list):
        return [int(x) for x in lbls if str(x).isdigit()]
    return []

# единообразно чистим и приводим к int
main_df["classification"] = main_df["classification"].apply(to_int_list)
main_df = main_df[main_df["classification"].map(len) > 0]

# теперь все элементы — int, ошибка сортировки исчезает
all_labels = sorted({l for sub in main_df["classification"] for l in sub})
label2id   = {lbl: i for i, lbl in enumerate(all_labels)}
num_labels = len(all_labels)


def to_multihot(label_list):
    vec = np.zeros(num_labels, dtype=np.float32)
    for lbl in label_list:
        if lbl in label2id: vec[label2id[lbl]] = 1.
    return vec

val_df["y_true"] = val_df["manual_labels"].apply(to_multihot)


In [6]:
# ---------- 6. загружаем best‑модель ----------------------------------------
device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model    = LongformerForSequenceClassification.from_pretrained("best_model").to(device)
tokenizer= LongformerTokenizerFast.from_pretrained("best_model")


In [7]:
# ---------- 7. токенизация validation‑набора --------------------------------
enc = tokenizer(
    val_df["text_str"].tolist(),
    max_length=16_384,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
dataset = torch.utils.data.TensorDataset(
    enc["input_ids"], enc["attention_mask"],
    torch.tensor(np.vstack(val_df["y_true"].values))
)
loader  = torch.utils.data.DataLoader(dataset, batch_size=3)



In [8]:
# ---------- 8. инференс ------------------------------------------------------
model.eval()
all_logits, all_true = [], []
with torch.no_grad():
    for ids, mask, y in loader:
        ids, mask = ids.to(device), mask.to(device)
        logits = model(ids, attention_mask=mask).logits
        all_logits.append(logits.cpu())
        all_true.append(y)

y_true = torch.vstack(all_true).numpy()
y_prob = torch.sigmoid(torch.vstack(all_logits)).numpy()
y_pred = (y_prob > 0.5).astype(int)



Initializing global attention on CLS token...


In [9]:
# ---------- 9. отчёт ---------------------------------------------------------
print(classification_report(
    y_true, y_pred,
    target_names=[str(lbl) for lbl in all_labels],
    zero_division=0
))
print("ROC‑AUC (micro):", roc_auc_score(y_true, y_prob, average="micro"))
print("ROC‑AUC (macro):", roc_auc_score(y_true, y_prob, average="macro"))

              precision    recall  f1-score   support

           0       0.88      0.28      0.42        50
           1       0.69      0.78      0.73        80
           2       0.83      0.75      0.79        64
           3       0.78      0.63      0.69        83
           4       0.93      0.67      0.78        55
           5       0.78      0.70      0.74       108
           6       1.00      0.19      0.33        72
           7       0.67      0.63      0.65       131

   micro avg       0.76      0.60      0.67       643
   macro avg       0.82      0.58      0.64       643
weighted avg       0.80      0.60      0.65       643
 samples avg       0.77      0.67      0.69       643

ROC‑AUC (micro): 0.9065219678727806
ROC‑AUC (macro): 0.9005317867692769
